# imports

In [1]:
import random
from collections import Counter
import tensorflow as tf
from tensorflow.keras.preprocessing import text,sequence

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout, LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [21]:
with open('/content/fr_articles.txt', encoding='utf8', errors='ignore') as f:
      text_corpus = f.read()
with open('/content/fr_summaries.txt', encoding='utf8', errors='ignore') as f:
      summary_corpus = f.read()

In [22]:
type(text_corpus), len(text_corpus), text_corpus[:300]

(str,
 16282170,
 "Jean-Jacques Schuhl, Gilles Leroy, Christian Gailly, Yasmina Khadra, James Ellroy, Amos Oz, V. S. Naipaul… Si la rentrée de janvier s'annonce sous les meilleurs auspices, force est de constater que, avec 491 romans (contre 558 en 2009), la tendance à la baisse enregistrée à l'automne s'accentue. Pri")

In [23]:
type(summary_corpus), len(summary_corpus), summary_corpus[:300]

(str,
 19322068,
 "Jean-Jacques Schuhl, Gilles Leroy, Christian Gailly, Yasmina Khadra, James Ellroy, Amos Oz, V. S. Naipaul… La rentrée de janvier s'annonce sous les meilleurs auspices.\nCette demande intervient une semaine après l'attaque terroriste manquée à bord d'un vol à destination des Etats-Unis par un jeune Ni")

# transformation du corpus de textes en textes et du corpus de résumés en résumés (séparateur \n)

In [24]:
texts = text_corpus.split('\n')[:400]
summaries = summary_corpus.split('\n')[:400]

In [25]:
len(texts), len(summaries)

(400, 400)

In [ ]:
for i in range(1):
    ind = random.randint(1,100)
    print('texte : {} : {} ' .format(ind,texts[ind]))
    print('résumé : {} : {} ' .format(ind,summaries[ind]))

texte : 64 :  Inventeur de l'extra-terrestre parasite et carnivore qui terrorise un vaisseau spatial dans Alien, mais aussi réalisateur d'un excellent film de zombies, Le Retour des morts-vivants, Dan O'Bannon est mort le 17 décembre 2009 à Santa Monica (Californie) de la maladie de Crohn, dont il souffrait depuis plusieurs années. Il avait 63 ans. Daniel O'Bannon est né en 1946 dans le Missouri. Il suit des études de cinéma à la University of Southern California, à Los Angeles, où il a pour condisciple John Carpenter. Avec le futur réalisateur de New York 1997, il écrit et conçoit un film de science-fiction bon marché, Dark Star. Dan O'Bannon en met au point les effets spéciaux. Le film sort en 1975. C'est un échec, mais il est remarqué par l'auteur et cinéaste Alejandro Jodorowsky qui prépare alors une adaptation de Dune, le classique de la science-fiction de Frank Herbert. Dan O'Bannon suit Jodorowsky à Paris où il rencontre les dessinateurs recrutés pour le projet, parmi lesquels l

# nombre de mots dans un texte
# nombre de mots uniques dans l'ensemble des textes
# les mots les plus représentés

In [ ]:
word = texts[9].split()
print(word)
print(f'nombre de mots : {len(word)}')
print(f'nombre de mots uniques : {len(Counter(word))}')

['Une', 'attaque', 'de', 'panique', 'constitue', 'un', 'trouble', 'de', "l'anxiété", 'sévère,', 'où', 'les', 'symptômes', 'mentaux', 'et', 'physiques', 'semblent', 'se', 'nourrir', 'mutuellement', ':', 'palpitations', 'cardiaques,', 'gêne', 'respiratoire,', 'transpiration,', 'vertige,', 'se', 'combiner', 'à', 'une', 'sensation', 'de', 'perte', 'de', 'contrôle', 'de', 'soi,', 'à', "l'impression", "d'être", 'en', 'train', 'de', 'mourir.', 'Si', 'les', 'origines', 'biologique', 'et', 'psychologique', 'de', 'ces', 'crises', 'sont', 'parfois', 'difficiles', 'à', 'démêler,', 'il', 'a', 'été', 'démontré', "qu'elles", 'peuvent', 'être', 'induites,', 'tant', 'chez', "l'homme", 'que', 'chez', "l'animal,", 'par', 'du', 'lactate', 'de', 'sodium', 'ou', 'des', 'inhalations', 'de', 'CO', '2', '.', 'Des', 'médicaments', 'permettent', 'de', 'les', 'juguler,', 'mais', 'ils', 'ne', 'sont', 'pas', 'sans', 'inconvénients', '(risque', 'de', 'dépendance).', 'Une', 'nouvelle', 'étude', 'illustre', 'le', 'rôl

In [ ]:
inverse_dico_counter = {}
for (k,v) in Counter(word).items():
    inverse_dico_counter[v] = k

most_freq_words = []
for i in sorted(inverse_dico_counter.keys(), reverse = True)[:10]:
    most_freq_words.append(inverse_dico_counter[i])

print(f'mots les plus fréquents : {most_freq_words}')

mots les plus fréquents : ['de', 'des', 'et', 'que', 'chez', 'la', 'qui', 'pour', 'récepteurs', 'Morin']


# polarité, subjectivité

In [ ]:
!pip install textblob-fr

In [ ]:
from textblob import TextBlob, Blobber
from textblob_fr import PatternTagger, PatternAnalyzer

tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

for i in range(1):
    ind = random.randint(1, 100)
    print('texte : {}, polarité : {}, subjectivité : {}'.format(ind, tb(texts[ind]).sentiment[0], tb(texts[ind]).sentiment[1]))
    print('résumé : {}, polarité : {}, subjectivité : {}'.format(ind, tb(summaries[ind]).sentiment[0], tb(summaries[ind]).sentiment[1]))

texte : 66, polarité : 0.10943014705882352, subjectivité : 0.2706433823529414
résumé : 66, polarité : 0.0, subjectivité : 0.0


In [ ]:
def polarity(text):
    if tb(text).sentiment[0] < 0:
        print(f'La polarité de ce texte est {tb(text).sentiment[0]} : ce texte est plus négatif que positif')
    elif tb(text).sentiment[0] > 0:
        print(f'La polarité de ce texte est {tb(text).sentiment[0]} : ce texte est plus positif que négatif')
    else :
        print(f'La polarité de ce texte est {tb(text).sentiment[0]} : ce texte est neutre, pas plus négatif que positif')
    return

In [ ]:
polarity(texts[9])

La polarité de ce texte est 0.06535714285714286 : ce texte est plus positif que négatif


In [ ]:
def subjectivity(text):
    if tb(text).sentiment[1] < 0.5:
        print(f'La subjectivité de ce texte est {tb(text).sentiment[1]} : ce texte est plus subjectif que factuel')
    elif tb(text).sentiment[1] > 0.5:
        print(f'La subjectivité de ce texte est {tb(text).sentiment[1]} : ce texte est plus subjectif que factuel')
    else :
        print(f'La subjectivité de ce texte est {tb(text).sentiment[1]} : ce texte est neutre, pas plus subjectif que factuel')
    return

In [ ]:
subjectivity(texts[9])

La subjectivité de ce texte est 0.2749999999999999 : ce texte est plus subjectif que factuel


# tag et mots "clés"

In [ ]:
# (texts[0]).tags

In [ ]:
import spacy.cli
spacy.cli.download("fr_core_news_md")

nlp = spacy.load("fr_core_news_md")

#text = nlp(texts[9])
#text_keywords = [token.text for token in text if token.pos_== 'NOUN' or token.pos_== 'PROPN' or token.pos_== 'VERB']

inverse_dico_counter = {}
for (k,v) in Counter(text_keywords).items():
    inverse_dico_counter[v] = k

most_freq_words = []
for i in sorted(inverse_dico_counter.keys(), reverse = True)[:10]:
    most_freq_words.append(inverse_dico_counter[i])

print(f'mots clés : {most_freq_words}')

In [ ]:
def keywords(text):
    text2 = nlp(text)
    text_keywords = [token.text for token in text2 if token.pos_== 'NOUN' or token.pos_== 'PROPN' or token.pos_== 'VERB']
    counter_words = Counter(text_keywords)
    most_freq_words = [word for word in counter_words.most_common(10)]
    most_freq_words_p = []
    for i in range(len(most_freq_words)):
        mfwp = most_freq_words[i][0]
        most_freq_words_p.append(mfwp)
    return print('mots clés :'), print(*most_freq_words_p, sep=', ')

In [ ]:
keywords(texts[9])

mots clés :
orexine, panique, rats, Johnson, -, trouble, anxiété, contrôle, homme, étude


(None, None)

In [ ]:
# def keywords(text):
    counter = {}
    text2 = nlp(text)
    text_keywords = [token.text for token in text if token.pos_== 'NOUN' or token.pos_== 'PROPN' or token.pos_== 'VERB']
    for word in text_keywords :
        if word not in counter :
            counter[word] = 0
        counter[word] += 1
        return counter

In [ ]:
import pandas as pd

def keywords2(text):
    text2 = nlp(text)
    text_keywords = [token.text for token in text2 if token.pos_== 'NOUN' or token.pos_== 'PROPN' or token.pos_== 'VERB']
    df = pd.DataFrame(text_keywords)
    l = list(dict(df.value_counts()[:10]).keys())
    most_freq_words_p = []
    for i in range(len(l)):
        mfwp= l[i][0]
        most_freq_words_p.append(mfwp)
    return print('mots clés :'), print(*most_freq_words_p, sep=', ')

In [ ]:
keywords2(texts[9])

mots clés :
orexine, panique, -, Johnson, rats, comportements, homme, constaté, pistes, collègues


(None, None)

# preprocess : tokenization and padding

In [26]:
stop_words = stopwords.words('french')
lemmatizer = WordNetLemmatizer()

def text_preprocessing(text):
    text_prep = []
    for i in range(len(text)):
      tp1 = text[i].lower()
      tp2 = word_tokenize(tp1)
      tp3 = [t for t in tp2 if t.isalpha()]
      tp4 = [t for t in tp3 if t not in stop_words]
      tp5 = [lemmatizer.lemmatize(w) for w in tp4]
      text_prep.append(tp5)
    return text_prep

In [27]:
texts_prep = text_preprocessing(texts)
summaries_prep = text_preprocessing(summaries)

In [28]:
def tokenize(text, summary):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(text)
    return tokenizer.texts_to_sequences(text), tokenizer, tokenizer.texts_to_sequences(summary)

In [29]:
tex_to_seq, tex_tokenizer, sum_to_seq = tokenize(texts_prep, summaries_prep)

In [30]:
len(tex_tokenizer.index_word)

19086

In [31]:
print(texts[9])
print(tex_to_seq[9])
print(tex_tokenizer.index_word)
print(summaries[9])
print(sum_to_seq[9])

Une attaque de panique constitue un trouble de l'anxiété sévère, où les symptômes mentaux et physiques semblent se nourrir mutuellement : palpitations cardiaques, gêne respiratoire, transpiration, vertige, se combiner à une sensation de perte de contrôle de soi, à l'impression d'être en train de mourir. Si les origines biologique et psychologique de ces crises sont parfois difficiles à démêler, il a été démontré qu'elles peuvent être induites, tant chez l'homme que chez l'animal, par du lactate de sodium ou des inhalations de CO 2 . Des médicaments permettent de les juguler, mais ils ne sont pas sans inconvénients (risque de dépendance). Une nouvelle étude illustre le rôle d'une hormone dans ce trouble anxieux, pouvant ouvrir des pistes thérapeutiques. Une équipe américano-suédoise a en effet constaté que certaines molécules inhibitrices de l'orexine (aussi nommée "hypocrétine") pouvaient réduire les comportements de panique observés chez des rats sélectionnés pour présenter ces troubl

In [ ]:
# function to pad texts

from os import truncate
from multiprocessing import Value
from tensorflow.keras.preprocessing import sequence

def pad(tex_seq, max_lenght) :
  return sequence.pad_sequences(tex_seq, value=0, padding='post', truncating='post', maxlen=max_lenght)

In [ ]:
tex_to_seq_pad = pad(tex_to_seq, max_lenght=None)
sum_to_seq_pad = pad(sum_to_seq, max_lenght=1064)

In [ ]:
tex_to_seq_pad, tex_to_seq_pad.shape

(array([[ 6593,  1960,  6594, ...,     0,     0,     0],
        [  362,    19,   619, ...,     0,     0,     0],
        [  347,   102,     1, ...,     0,     0,     0],
        ...,
        [   89,  6583,   204, ...,     0,     0,     0],
        [ 8753, 19042,     3, ...,     0,     0,     0],
        [  115,  2946, 19062, ...,     0,     0,     0]], dtype=int32),
 (400, 1064))

In [ ]:
sum_to_seq_pad, sum_to_seq_pad.shape

(array([[6593, 1960, 6594, ...,    0,    0,    0],
        [   3,  231, 1880, ...,    0,    0,    0],
        [ 347,  102,    1, ...,    0,    0,    0],
        ...,
        [  20,  102,  339, ...,    0,    0,    0],
        [  76, 2155, 9824, ...,    0,    0,    0],
        [ 178,   94,    1, ...,    0,    0,    0]], dtype=int32),
 (400, 1064))

# turn logits from a neural network into text using the tokenizer

In [ ]:
import numpy as np
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'
    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


# model: text to summary in French

In [ ]:
def sum_model(input_shape, dic_size):

  learning_rate = 0.005

  model = Sequential()
  model.add(Embedding(dic_size, 300, input_length=input_shape[1]))
  model.add(LSTM(300, return_sequences=True))
  model.add(Dropout(0.2))
  model.add(Dense(dic_size, activation='softmax'))

  model.compile(optimizer=Adam(learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

  return model

In [ ]:
# train the model
mdl = sum_model(tex_to_seq_pad.shape, len(tex_tokenizer.index_word)+1)

a = mdl.fit(tex_to_seq_pad, sum_to_seq_pad, validation_split=0.2, epochs=1, batch_size=16)

In [ ]:

print("prediction:")
print(logits_to_text(mdl.predict(tex_to_seq_pad[0:1])[0], tex_tokenizer))

print("\ncorrect summary:")
print(summaries[0:1])

print("\noriginal text:")
print(texts[0:1])

# model 2

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Dense, Attention, Dot, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np

# Define your input sequence length and vocabulary size
input_seq_length = tex_to_seq_pad.shape[1]
output_seq_length = sum_to_seq_pad.shape[1]
vocab_size = len(tex_tokenizer.index_word)

# Define encoder
encoder_inputs = Input(shape=(input_seq_length, vocab_size))
encoder = LSTM(units=300, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# Define decoder with attention
decoder_inputs = Input(shape=(output_seq_length, vocab_size))
decoder_lstm = LSTM(units=300, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=[state_h, state_c])
attention_layer = Attention()([decoder_outputs, encoder_outputs])
context_vector = Dot(axes=[2, 2])([attention_layer, encoder_outputs])
decoder_combined_context = Concatenate(axis=-1)([context_vector, decoder_outputs])
output = Dense(vocab_size, activation='softmax')(decoder_combined_context)

# Create the model
model = Model([encoder_inputs, decoder_inputs], output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# One-hot encode your target sequences
num_decoder_tokens = len(tex_tokenizer.word_index) + 1
decoder_target_data = np.zeros((len(sum_to_seq_pad), output_seq_length, num_decoder_tokens), dtype='float32')
for i, sequence in enumerate(sum_to_seq_pad):
    for t, word_index in enumerate(sequence):
        if t > 0:
            decoder_target_data[i, t - 1, word_index] = 1.0

# Train the model
history = model.fit(
    [tex_to_seq_pad, sum_to_seq_pad[:, :-1]],  # Use the summary input as input to the decoder
    decoder_target_data,
    batch_size=64,
    epochs=10,
    validation_split=0.2
)

# ...

# model 3

In [ ]:
! pip install sentencepiece

In [ ]:
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.4 MB/s eta 0:00:00


In [ ]:
import torch
import sentencepiece as spm
from transformers import CamembertTokenizer, CamembertModel
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
model = CamembertModel.from_pretrained('camembert-base')
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

In [ ]:
def summary(text):

  # preprocessing
  sentences = sent_tokenize(text)
  tokenized_sentences = [tokenizer.encode(sent, add_special_tokens=True) for sent in sentences]

  # padding, encoding
  max_len = 0
  for i in tokenized_sentences:
    if len(i) > max_len:
      max_len = len(i)

  padded_sentences = []
  for i in tokenized_sentences:
    while len(i) < max_len:
      i.append(0)
    padded_sentences.append(i)

  input_ids = torch.tensor(padded_sentences)

  # embedding
  with torch.no_grad():
    last_hidden_states = model(input_ids)[0]

  sentence_embeddings = []
  for i in range(len(sentences)):
    sentence_embeddings.append(torch.mean(last_hidden_states[i], dim=0).numpy())

  # summarizing
  similarity_matrix = cosine_similarity(sentence_embeddings)

  num_sentences = 2
  summary_sentences = []
  for i in range(num_sentences):
    sentence_scores = list(enumerate(similarity_matrix[i]))
    sentence_scores = sorted(sentence_scores, key=lambda x: x[1], reverse=True)
    summary_sentences.append(sentences[sentence_scores[1][0]])

  summary = ' '.join(summary_sentences)
  return summary


In [ ]:
summary(texts[15])

'Vers la fin de la prochaine décennie, Moscou entend faire du Grand Nord la "base stratégique de (ses) ressources naturelles", au risque de sacrifier encore davantage l\'écosystème déjà affaibli de la région. Une ONG norvégienne, Bellona, a lancé en vain un signal d\'alarme : les installations qui accueillent les vieux sous-marins sont également désuètes et pourraient facilement contaminer l\'air et l\'eau environnants.'

In [ ]:
texts[15]

'Les autorités russes n\'ont pas voulu participer à l\'étude menée par l\'association Robin des bois, bien que - ou peut-être parce que - la situation environnementale du nord du pays est particulièrement dégradée. Le Grand Nord russe souffre en effet de plusieurs maux : pollution industrielle de l\'air et de l\'eau en raison de l\'exploitation de matières premières, et radioactivité et pollution nucléaire là où des installations militaires ont été déployées. Le problème est d\'autant plus grave que la région est assez peuplée : sur quatre millions de terriens vivant dans la région arctique, deux millions vivent en Russie. Mourmansk (300 000 habitants) et Norilsk (200 000 habitants) sont les plus grandes villes du monde établies au nord du cercle arctique. La ville de Norilsk constitue un exemple du désastre écologique en cours ; l\'ONG américaine Blacksmith Institute en a d\'ailleurs fait l\'une des dix villes les plus polluées du monde en 2008. Centre d\'extraction de nickel, mais au

# model 4

In [ ]:
def sum(text):

    text2 = nlp(text)
    text_keywords = [token.text for token in text2 if token.pos_== 'NOUN' or token.pos_== 'PROPN']

    counter_words = Counter(text_keywords)
    most_freq_words = [word for word in counter_words.most_common(3)]

    most_freq_words_p = []
    for i in range(len(most_freq_words)):
        mfwp = most_freq_words[i][0]
        most_freq_words_p.append(mfwp)

    sentences = sent_tokenize(text)
    summary= []
    for sent in sentences:
      for word in sent.split():
        if word in most_freq_words_p and sent not in summary:
            summary.append(sent)
        else :
          None

    return summary

In [ ]:
a = sum(texts[192])
a

['Alex Ferguson dit la même chose à Manchester United, pourtant privé des trois quarts de sa défense pour cause de blessures.',
 'DZEKO AFFOLE LE MILAN Liverpool, éliminé en Ligue des champions et distancé en Premier League, pourrait chercher des renforts, mais les propriétaires américains du club ne semblent pas prêts à investir, hormis la bonne affaire van Nistelrooy, dont le salaire risque toutefois de poser problème, une nouveauté pour un club du Big Four.',
 "Les Allemands devraient négocier le transfert pour la prochaine intersaison, le joueur lui-même n'ayant pas intérêt à changer de club à six mois de la Coupe du monde, alors qu'il est toujours convalescent.",
 "Malgré quelques trous d'air, la défense a donné satisfaction, et Deschamps cherche surtout un renfort offensif (Issiar Dia ?",
 "A Lyon, c'est la défense qui prend l'eau (13e de L1), en même temps que la réputation de Claude Puel, entraîneur contesté.",
 'Le président Aulas pourrait recruter Younès Kaboul, qui peut joue

In [ ]:
len(texts[192].split())

914

In [ ]:
num_words = []
for i in range(len(a)):
  nw = len(a[i].split())
  num_words.append(nw)

print(num_words)

[21, 53, 33, 19, 22, 20]
